In [ ]:
import pandas as pd

data = pd.read_csv('stock_details_5_years.csv')

data

In [ ]:
for campany, group in data.groupby('Company'):
    print(f"tackling: {campany}")
    group.to_csv(f"splited/{campany}.csv", index=False)
    

In [5]:
import os

def cal_ad_cap(input_folder, output_folder):
    # 입력 폴더 내 모든 파일을 가져옵니다.
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".csv"):
            input_file_path = os.path.join(input_folder, file_name)
            
            # CSV 파일 로드
            data = pd.read_csv(input_file_path)
            
            # 시가총액을 1로 초기화 (Close Price를 기준으로 시작)
            initial_market_cap = 1.0
            data['Adjusted Market Cap'] = initial_market_cap

            # 첫 번째 종가를 기준으로 시가총액 추이를 계산합니다.
            first_close_price = data.iloc[0]['Close']

            # Adjusted Market Cap 계산
            for i in range(1, len(data)):
                # 전날 주가와 주식 분할 정보를 가져옵니다.
                previous_close = data.iloc[i - 1]['Close']
                stock_splits = data.iloc[i]['Stock Splits']
                
                # 주식 분할이 발생하면 주식 수를 조정합니다.
                if stock_splits != 0:
                    first_close_price /= stock_splits

                # Adjusted Market Cap 계산: (현재 종가 / 첫 번째 종가) * 1.0
                data.at[i, 'Adjusted Market Cap'] = (data.iloc[i]['Close'] / first_close_price) * initial_market_cap

            data = data[['Date', 'Adjusted Market Cap']]
            output_file_path = os.path.join(output_folder, f"{file_name}")
            data.to_csv(output_file_path, index=False)
            print(f"Processed {file_name}, saved as {output_file_path}")


In [ ]:
cal_ad_cap('splited', 'adjusted')

In [ ]:
import pandas as pd
import os

def merge_ad_data(adjusted_folder, output_file):
    merged_data = pd.DataFrame()

    # adjusted 폴더 내 모든 파일을 가져옵니다.
    for file_name in os.listdir(adjusted_folder):
        if file_name.endswith(".csv"):
            file_path = os.path.join(adjusted_folder, file_name)
            
            # CSV 파일 로드
            data = pd.read_csv(file_path)
            
            # 기업 이름을 파일명에서 추출 (확장자를 제거)
            company_name = os.path.splitext(file_name)[0]
            
            # 'Date' 열을 인덱스로 설정하고 'Adjusted Market Cap'만 남기고 병합 준비
            data = data[['Date', 'Adjusted Market Cap']].rename(columns={'Adjusted Market Cap': company_name})
            
            if merged_data.empty:
                # 첫 번째 파일일 경우 'Date' 열을 포함한 데이터프레임 생성
                merged_data = data
            else:
                # 이후 파일은 'Date'를 기준으로 병합
                merged_data = pd.merge(merged_data, data, on='Date', how='outer')

    # 'Date' 열을 맨 앞으로 두고, 나머지 열을 알파벳 순으로 정렬
    columns_sorted = ['Date'] + sorted(merged_data.columns.drop('Date'))
    merged_data = merged_data[columns_sorted]
    
    # 병합된 데이터 저장
    merged_data.to_csv(output_file, index=False)
    print(f"Merged data saved as {output_file}")

# 함수 호출 예시: adjusted 폴더의 데이터를 병합하여 output.csv로 저장
adjusted_folder = 'adjusted'
output_file = 'merged_adjusted_market_cap.csv'
merge_ad_data(adjusted_folder, output_file)


In [12]:
df = pd.read_csv('merged_adjusted_market_cap.csv')

In [ ]:
df_m7 = df[["AAPL","MSFT", "NVDA", "GOOGL", "AMZN","META", "TSLA" ]]